In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test_data.parquet
/kaggle/input/train_features_merged.parquet
/kaggle/input/train_data_cleaned.parquet
/kaggle/input/events_train.parquet
/kaggle/input/train_final.parquet
/kaggle/input/df_trainn.parquet
/kaggle/input/df_test.parquet
/kaggle/input/events_features_test.parquet
/kaggle/input/all_features_train.parquet
/kaggle/input/test_final.parquet
/kaggle/input/events_features_train.parquet
/kaggle/input/trans_features_train.parquet
/kaggle/input/add_trans.parquet
/kaggle/input/df_testt.parquet
/kaggle/input/offer_clean.parquet
/kaggle/input/id3_train_extended.parquet
/kaggle/input/add_event_cleaned.parquet
/kaggle/input/test_features_merged.parquet
/kaggle/input/events_test.parquet
/kaggle/input/id3_test_extended.parquet
/kaggle/input/all_features_test.parquet
/kaggle/input/test_data_cleaned.parquet
/kaggle/input/test_fml.parquet
/kaggle/input/trans_features_test.parquet
/kaggle/input/train_data.parquet
/kaggle/input/df_train.parquet


In [2]:
import pandas as pd
import kagglehub

# Correct handle
path = kagglehub.dataset_download("aryansachannn/amex-cleaned-datasets")

print("✅ Dataset mounted at:", path)
# Load the files from the mounted input dataset
df_train = pd.read_parquet(f"{path}/train_features_merged.parquet")
df_test = pd.read_parquet(f"{path}/test_features_merged.parquet")



✅ Dataset mounted at: /kaggle/input/


In [3]:
df_test = df_test.rename(columns={
    'median_imp_hour': 'median_imp_hour_y',
    'median_imp_month': 'median_imp_month_y'
})


In [4]:
import numpy as np

# STEP 1: Drop constant columns
constant_cols = [col for col in df_train.columns if df_train[col].nunique() <= 1]
df_train.drop(columns=constant_cols, inplace=True)
df_test.drop(columns=constant_cols, errors='ignore', inplace=True)
print(f"✅ Dropped {len(constant_cols)} constant columns.")

# STEP 2: Drop columns with too many missing values (e.g., > 99%)
null_thresh = 0.99
null_cols = [col for col in df_train.columns if df_train[col].isnull().mean() > null_thresh]
df_train.drop(columns=null_cols, inplace=True)
df_test.drop(columns=null_cols, errors='ignore', inplace=True)
print(f"✅ Dropped {len(null_cols)} mostly-null columns.")

# STEP 3: Drop features with very low correlation with y
numeric_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'y']

correlations = df_train[numeric_cols].corrwith(df_train['y']).abs()
low_corr_cols = correlations[correlations < 0.01].index.tolist()

df_train.drop(columns=low_corr_cols, inplace=True)
df_test.drop(columns=low_corr_cols, errors='ignore', inplace=True)
print(f"✅ Dropped {len(low_corr_cols)} low-correlation columns.")

# ✅ Fix: Recompute numeric columns after dropping
numeric_cols = df_train.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'y']

# STEP 4: Drop highly correlated duplicate features (threshold > 0.98)
corr_matrix = df_train[numeric_cols].corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_pairs = [col for col in upper_triangle.columns if any(upper_triangle[col] > 0.98)]

df_train.drop(columns=high_corr_pairs, inplace=True)
df_test.drop(columns=high_corr_pairs, errors='ignore', inplace=True)
print(f"✅ Dropped {len(high_corr_pairs)} highly redundant features.")


✅ Dropped 72 constant columns.
✅ Dropped 0 mostly-null columns.
✅ Dropped 138 low-correlation columns.


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


✅ Dropped 20 highly redundant features.


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# Expected output: Num GPUs Available:  2

2025-07-20 11:13:34.451053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753010014.841265      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753010014.946717      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  2


In [6]:
!pip install optuna


In [7]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.metrics import average_precision_score
from xgboost import XGBClassifier, DMatrix
from sklearn.model_selection import train_test_split
import time

#  Load train/test data

X = df_train.drop(columns=['y'])
y = df_train['y'].astype(int)  


#  Drop object columns not needed for training
X = X.drop(columns=['id1', 'id2', 'id3', 'id4', 'id5', 'id9', 'id12', 'id13'])
df_test_features = df_test.drop(columns=['id1', 'id2', 'id3', 'id4', 'id5', 'id9', 'id12', 'id13'])

#  Enable categorical support
X = X.copy()
df_test_features = df_test_features.copy()
for col in X.select_dtypes(include='category').columns:
    X[col] = X[col].astype('category')
    df_test_features[col] = df_test_features[col].astype('category')

#  Split train/valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    elapsed = time.time() - start_time
    mins = elapsed // 60
    secs = int(elapsed % 60)

    print(f"\n Trial {trial.number} |  Elapsed Time: {int(mins)}m {secs}s")

    params = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'eval_metric': 'map',
        'tree_method': 'hist',
        'enable_categorical': True,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_valid)[:, 1]
    score = average_precision_score(y_valid, preds)

    print(f" Trial {trial.number} complete | AP Score: {score:.5f}")
    return score

# Optuna study with timeout (30 minutes = 1800 seconds)
start_time = time.time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=9000)  # runs for 2.5 hours


print("✅ Best trial:")
print(study.best_trial)

#  Retrain on full training data with best params
final_model = XGBClassifier(**study.best_params, 
                            objective='binary:logistic',
                            eval_metric='map',
                            tree_method='hist',
                            enable_categorical=True)
final_model.fit(X, y)

#  Predict on test
test_preds = final_model.predict_proba(df_test_features)[:, 1]

#  Format submission
submission = df_test[['id1', 'id2', 'id3', 'id5']].copy()
submission['pred'] = test_preds
submission = submission[['id1', 'id2', 'id3', 'id5', 'pred']]

team_name = "Amex_Predators"
submission_filename = f"r2_optuna_submission_{team_name}.csv"
submission.to_csv(submission_filename, index=False)

print(f"\n Final submission saved as: {submission_filename}")
print(f" Total tuning time: {(time.time() - start_time)/60:.2f} minutes")


[I 2025-07-20 11:14:05,814] A new study created in memory with name: no-name-e1e7ef9b-9fd4-4df4-8fcc-1cc3716f2244



🧪 Trial 0 | ⏱️ Elapsed Time: 0m 0s


[I 2025-07-20 11:14:47,829] Trial 0 finished with value: 0.7938137213043248 and parameters: {'learning_rate': 0.15358667773774864, 'max_depth': 10, 'min_child_weight': 1, 'gamma': 2.6844141757626594, 'subsample': 0.991506109452495, 'colsample_bytree': 0.9526776677757143, 'n_estimators': 127}. Best is trial 0 with value: 0.7938137213043248.


✅ Trial 0 complete | AP Score: 0.79381

🧪 Trial 1 | ⏱️ Elapsed Time: 0m 42s


[I 2025-07-20 11:15:21,070] Trial 1 finished with value: 0.7802229369360644 and parameters: {'learning_rate': 0.14294020046629524, 'max_depth': 7, 'min_child_weight': 1, 'gamma': 4.952283081824332, 'subsample': 0.5531292156448049, 'colsample_bytree': 0.6507769998799199, 'n_estimators': 104}. Best is trial 0 with value: 0.7938137213043248.


...

[I 2025-07-20 11:57:09,904] Trial 36 finished with value: 0.7956243437079308 and parameters: {'learning_rate': 0.11651826260545321, 'max_depth': 7, 'min_child_weight': 3, 'gamma': 1.6184586956596263, 'subsample': 0.920947397864311, 'colsample_bytree': 0.5136165030165472, 'n_estimators': 274}. Best is trial 27 with value: 0.8011831791992213.


✅ Trial 36 complete | AP Score: 0.79562

🧪 Trial 37 | ⏱️ Elapsed Time: 43m 4s


[I 2025-07-20 11:58:06,837] Trial 37 finished with value: 0.796180013112249 and parameters: {'learning_rate': 0.15351494525167528, 'max_depth': 9, 'min_child_weight': 4, 'gamma': 2.7640477451481944, 'subsample': 0.9477716032072947, 'colsample_bytree': 0.5680441605292347, 'n_estimators': 300}. Best is trial 27 with value: 0.8011831791992213.


✅ Trial 37 complete | AP Score: 0.79618

🧪 Trial 38 | ⏱️ Elapsed Time: 44m 1s


[I 2025-07-20 11:59:12,743] Trial 38 finished with value: 0.7934540630627773 and parameters: {'learning_rate': 0.13473904728564318, 'max_depth': 8, 'min_child_weight': 1, 'gamma': 3.3821797764735733, 'subsample': 0.594453816221447, 'colsample_bytree': 0.5844874957672165, 'n_estimators': 236}. Best is trial 27 with value: 0.8011831791992213.
